In [ ]:
#imports

import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display
from itertools import zip_longest

In [ ]:
#set up

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY') 

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

In [ ]:
#Connect to OpenAI cliet library

openai = OpenAI()

anthropic_url = "https://api.anthropic.com/v1/"
ollama_url = "http://localhost:11434/v1"

anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
ollama = OpenAI(api_key="ollama", base_url=ollama_url)


In [ ]:
gpt_model = "gpt-4.1-mini"
claude_model = "claude-haiku-4-5"
ollama_model = "gpt-oss:20b"

gpt_system_prompt = (
    "You are a chatbot, named Trump."
    "You communicate and think like very experienced businessman. You are very straightforward and pragmatic."
    "You are in a chat with chatbot Nastya and chatbot Sheldon."
    "Here is the conversation so far:\n"
    "{conversation_so_far}\n"
    "Now with this, respond with what you would like to say next, as Trump."
)

claude_system_prompt = (
    "You are a chat bot, named Nastya."
    "You communicate like very infant person. You think you are the center of the world and everybody owe you"
    "You are in a chat with chatbot Trump and chatbot Sheldon."
    "Here is the conversation so far:\n"
    "{conversation_so_far}\n"
    "Now with this, respond with what you would like to say next, as Nastya."
)

ollama_system_prompt = (
    "You are a chatbot, named Sheldon."
    "You communicate like a snarky scientist. You respect only facts, logic and scientific approach."
    "You are in a chat with chatbot Trump and chatbot Nastya."
    "Here is the conversation so far:\n"
    "{conversation_so_far}\n"
    "Now with this, respond with what you would like to say next, as Sheldon."
)

gpt_messages = ["I wonder if I can earn money by selling air"]
claude_messages = ["What a selfish question you are asking"]
ollama_messages = ["It makes no sense"]

conversation_so_far = [
    f"Trump: {gpt_messages[0]}",
    f"Nastya: {claude_messages[0]}",
    f"Sheldon: {ollama_messages[0]}",
]

def build_messages(system_prompt):
    history = "\n".join(conversation_so_far)
    return [
        {"role": "system", "content": system_prompt.format(conversation_so_far=history)},
        {"role": "user", "content": "Continue the conversation with your next reply."}
    ]


In [ ]:

def call_gpt():
    messages =build_messages(gpt_system_prompt)
    response = openai.chat.completions.create(model=gpt_model, messages=messages)
    text = (response.choices[0].message.content or "").strip()
    conversation_so_far.append(f"Trump: {text}")
    return text
    

In [ ]:

def call_claude():
    messages =build_messages(claude_system_prompt)
    response = anthropic.chat.completions.create(model=claude_model, messages=messages)
    text = (response.choices[0].message.content or "").strip()
    conversation_so_far.append(f"Nastya: {text}")
    return text
    

In [ ]:

def call_ollama():
    messages =build_messages(ollama_system_prompt)
    response = ollama.chat.completions.create(model=ollama_model, messages=messages)
    text = (response.choices[0].message.content or "").strip()
    conversation_so_far.append(f"Sheldon: {text}")
    return text
    

In [ ]:
display(Markdown(conversation_so_far))

#Conversation

for i in range(5):
    gpt_next = call_gpt()
    display(Markdown(f"### GPT:\n{gpt_next}"))

    claude_next = call_claude()
    display(Markdown(f"### Claude:\n{claude_next}"))

    ollama_next = call_ollama()
    display(Markdown(f"### Ollama:\n{ollama_next}"))